In [1]:
import pandas as pd

In [2]:
# read the excel file and see the sheet names
xl_files = pd.ExcelFile(r'..\landing_zone\2026\Hyundia\2025-11-1 HACC MAF - 20251113.xlsx')

In [3]:
target_sheets = "Master"
if target_sheets in xl_files.sheet_names:
    hyundai_master_sheet_df = pd.read_excel(r"..\landing_zone\2026\Hyundia\2025-11-1 HACC MAF - 20251113.xlsx", sheet_name=target_sheets, skiprows=1)


In [4]:
# load Hyundia model db

hyundai_model_db_df = pd.read_csv(r"..\database\dbs\Hyundai_models_db.csv")

In [5]:
hyundai_master_sheet_df.columns

Index(['Model \nYear \nFrom', 'Model \nYear \nTo', 'Model', 'Trim 1', 'Trim 2',
       'Trim 3', 'Trim 4', 'Trim 5', 'Trim 6', 'Trim 7', 'Trim 8',
       'Accessory \nCategory (EN)', 'Accessory Group (EN)',
       'Accessory Description (EN)', 'Accessory \nCategory (FR)',
       'Accessory Group (FR)', 'Accessory Description (FR)', 'Part\nNumber',
       'DNET', 'MSRP', 'Labour Rate', 'Suggested Labour Hours',
       'Suggested Base Retail Price w/labour', 'Yearly Rate', 'Weekly Rate',
       '#of Payments', 'Weekly Payment', 'PKG Code', 'Comments (EN)',
       'Comments (FR)', 'New Part SOS', 'Outgoing Part Flag', 'MAF Update'],
      dtype='object')

In [6]:
key_columns_to_keep = ['Model \nYear \nFrom', 'Model', 'Trim 1', 'Trim 2',
       'Trim 3', 'Trim 4', 'Trim 5', 'Trim 6', 'Trim 7', 'Trim 8',
       'Accessory Description (EN)', 'Accessory Description (FR)',
       'Part\nNumber', 'DNET', 'MSRP', 'Labour Rate', 'Suggested Labour Hours',
       'Suggested Base Retail Price w/labour', 'Comments (EN)', 'Comments (FR)',]

In [7]:
hyundai_master_sheet_df = hyundai_master_sheet_df[key_columns_to_keep]

In [8]:
hyundai_master_sheet_df.columns

Index(['Model \nYear \nFrom', 'Model', 'Trim 1', 'Trim 2', 'Trim 3', 'Trim 4',
       'Trim 5', 'Trim 6', 'Trim 7', 'Trim 8', 'Accessory Description (EN)',
       'Accessory Description (FR)', 'Part\nNumber', 'DNET', 'MSRP',
       'Labour Rate', 'Suggested Labour Hours',
       'Suggested Base Retail Price w/labour', 'Comments (EN)',
       'Comments (FR)'],
      dtype='object')

In [9]:
hyundai_master_sheet_df = hyundai_master_sheet_df.dropna(axis=1, how='all')

In [10]:
hyundai_master_sheet_df.columns

Index(['Model \nYear \nFrom', 'Model', 'Trim 1', 'Trim 2', 'Trim 3', 'Trim 4',
       'Trim 5', 'Trim 6', 'Trim 7', 'Accessory Description (EN)',
       'Accessory Description (FR)', 'Part\nNumber', 'DNET', 'MSRP',
       'Labour Rate', 'Suggested Labour Hours',
       'Suggested Base Retail Price w/labour', 'Comments (EN)',
       'Comments (FR)'],
      dtype='object')

In [11]:
column_names_to_change = {
    'Model \nYear \nFrom': "Year", 
    'Part\nNumber':'PartNumber', 
    'Suggested Labour Hours':'Labour Hours',
    'Suggested Base Retail Price w/labour':'MSPR w/labour'}

In [12]:
standard_labour_rate = hyundai_master_sheet_df["Labour Rate"].unique().tolist()[0]

In [13]:
hyundai_master_sheet_df.rename(columns=column_names_to_change, inplace=True)

In [14]:
hyundai_master_sheet_df.columns

Index(['Year', 'Model', 'Trim 1', 'Trim 2', 'Trim 3', 'Trim 4', 'Trim 5',
       'Trim 6', 'Trim 7', 'Accessory Description (EN)',
       'Accessory Description (FR)', 'PartNumber', 'DNET', 'MSRP',
       'Labour Rate', 'Labour Hours', 'MSPR w/labour', 'Comments (EN)',
       'Comments (FR)'],
      dtype='object')

In [15]:
def show_unique_values(df, columns):
    """
    Display unique values for the specified columns in a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to analyze.
    columns (list): List of column names to check.
    
    Returns:
    dict: A dictionary with column names as keys and unique values as lists.
    """
    unique_dict = {}
    for col in columns:
        if col in df.columns:
            unique_dict[col] = df[col].unique().tolist()
            print(f"{col}: {df[col].unique().tolist()}")
        else:
            unique_dict[col] = "Column not found"
    return unique_dict


result = show_unique_values(hyundai_master_sheet_df, ['Trim 1', 'Trim 2',
       'Trim 3', 'Trim 4', 'Trim 5', 'Trim 6', 'Trim 7'])


Trim 1: ['Pref', nan, 'Ess', 'Advanced', '2.5T Advanced', 'Prestige', '2.0T Advanced', '2.5T Adv', '3.5T e-SC', 'Pref HEV', 'N', '2.5T Advanced 5P', 'Pref*', '3.5T Coupe', 'Pref SR', '3.5T e-SC Prestige', 'Pref Trend', 'Pref LR']
Trim 2: ['Urban', nan, 'Trend', 'Pref', 'Ult', 'Lux', 'Performance', '2.5T Advanced Plus', '2.5T Prestige', '2.0T Prestige', '3.5T Sport Plus', '3.5T Advanced', 'Trend AWD', 'Pref HEV w/ Trend', '2.5T Advanced Tech Pkg 5P', 'Trend*', 'XRT', 'Pref LR', '2.5T Advanced Tech', 'Prestige', '3.5T e-SC Prestige Black', '3.5T Prestige Black']
Trim 3: ['Calli', nan, 'N-Line', 'Ult', 'Lux', 'Trend', '2.5T Prestige', '3.3T Advanced', '3.5T Prestige', 'XRT', '3.5T Advanced Tech Pkg 7P', '3.5T Sport', '3.5T Advanced Tech', '3.3T Sport', 'Calli ICE', 'Lux HEV']
Trim 4: [nan, 'Lux', 'N-Line', '3.5T Sport', '3.3T Sport', '3.5T Prestige 7P', 'Sport', 'N-Line Ult', 'Ult Calli', '3.5T Prestige', 'Lux HEV', 'HEV', 'Calli HEV']
Trim 5: [nan, 'Ult HEV', 'HEV', 'N-Line Ult', '3.5T S

In [16]:
unique_years = hyundai_master_sheet_df['Year'].unique()

In [17]:
unique_hyundai_models = hyundai_master_sheet_df['Model'].unique()

In [18]:
trim_columns = [col for col in hyundai_master_sheet_df.columns if "trim" in col.lower()]

In [19]:
trim_columns

['Trim 1', 'Trim 2', 'Trim 3', 'Trim 4', 'Trim 5', 'Trim 6', 'Trim 7']

In [20]:
hyundai_master_sheet_df.columns

Index(['Year', 'Model', 'Trim 1', 'Trim 2', 'Trim 3', 'Trim 4', 'Trim 5',
       'Trim 6', 'Trim 7', 'Accessory Description (EN)',
       'Accessory Description (FR)', 'PartNumber', 'DNET', 'MSRP',
       'Labour Rate', 'Labour Hours', 'MSPR w/labour', 'Comments (EN)',
       'Comments (FR)'],
      dtype='object')

In [21]:
# dataset_summary = {}
# year_model_trim_dict = {}
# counter = 0
# for year in hyundai_master_sheet_df['Year'].unique():
#     for model in hyundai_master_sheet_df['Model'].unique():
        
#         model_df = hyundai_master_sheet_df[(hyundai_master_sheet_df['Year'] == year) & (hyundai_master_sheet_df['Model'] == model)]
#         filetered_columns = [col for col in model_df.columns if "trim" in col.lower()]
#         for col in filetered_columns:  
#             # dataset_summary[(year, model)] = len(model_df)
#             trim = model_df[col].dropna().unique().tolist()
#             if len(trim) > 0:
#                 trim_name = trim[0]
#                 # print(f"Year: {year}, Model: {model}, Trim: {trim_name}")
#                 year_model_trim_dict[(year, model, trim)] = model_df[(model_df['Year'] == year) & (model_df['Model'] == model) & (model_df[col] == trim_name)]
#                 # dataset_summary[(year, model, trim)] = len(year_model_trim_dict[(year, model, trim)])


In [22]:
standard_column_names = ['Year', 'Model', 'Trim', 'Description', 'PartNumber',
       'DNET', 'MSRP', 'Labour Rate', 'Hours', 'MSPR w/labour',
       'Comments']

In [23]:
column_to_match_rate_importer = ['Year', 'Model', 'Trim', 'Description', 'Part', 'Cost', 'Price', 'Hours' 'Comments']

In [52]:
tag_keywords = {    
    "Urban": ["Urban"],
    "Pref":["Preferred"],
    "Pref Trend":["Preferred Trend"],
    "Calli" : ["Calligraphy"],
    "Calli ICE":["Calligraphy",],
    "Lux HEV": ["Luxury HEV"],
    "XRT" : ["XRT"],
    "Trend": ["Trend"], # Tucson
    'N-Line': ["N-Line"], 
    'Lux':["Luxury"], 
    'Ult HEV':['Ultimate HEV'], 
    'Ult PHEV':['Ultimate PHEV'],
    'Ess': ["Essential"],  # Essential
    "N": ['N '],

}


def get_model_number_by_year_model_with_trim_key_word(year, model, kwd):
    
    pattern = ''.join(f'(?=.*{kw})' for kw in kwd)  # constructing search keywords pattern 
    # print(pattern)
    # search query
    res_df = hyundai_model_db_df[
        (1==1)
        &(hyundai_model_db_df["Year"] == year) 
    & (hyundai_model_db_df["Model"] == model) 
    & (hyundai_model_db_df["Trim"].str.contains(pattern, case=False, na=False))
    ]

    return  res_df

def get_tags_by_model_and_trim(trim):
    return tag_keywords[trim]
    
def search_hyundai_db_by_model_year_trim(year, model, trim):
    tags = get_tags_by_model_and_trim(trim)
    
    res = get_model_number_by_year_model_with_trim_key_word(year, model, tags)
    
    return res["ModelNumber"].tolist()


In [56]:
search_hyundai_db_by_model_year_trim(2025, "Elantra", 'Ess')

['EL74IF20A100']

In [ ]:
dataset_summary = pd.DataFrame(columns=['Year', 'Model', 'Trim', 'Record_Count'])
year_model_trim_dict = {}



for year in hyundai_master_sheet_df['Year'].unique():
    for model in hyundai_master_sheet_df['Model'].unique():
        
        model_df = hyundai_master_sheet_df[
            (hyundai_master_sheet_df['Year'] == year) &
            (hyundai_master_sheet_df['Model'] == model)
        ]
        
        filtered_columns = [col for col in model_df.columns if "trim" in col.lower()]
        
        for col in filtered_columns:
            trims = model_df[col].dropna().unique().tolist()
            if len(trims) == 0:
                continue            
            trim_name = trims[0]
            
            if trim_name:
                
                # create teh subset for each trim
                subset_df = model_df[model_df[col] == trim_name]
                
                copy_filtered_columns = [c for c in subset_df.columns if "trim" in c.lower()]  # grab all trim columns in the subset
                
                copy_filtered_columns.remove(col)  # drop the current trim column
 
                subset_df = subset_df.drop(columns=copy_filtered_columns)  #  drop other trim columns
                subset_df.rename(columns={col: 'Trim'}, inplace=True)       # rename the current trim column to 'Trim' for clarity
                
                # create copy of the subset dataframe and store both in the dist as english and french versions
                # Identify columns containing "(EN)"
                English_columns = [col for col in hyundai_master_sheet_df.columns if "(EN)" in col]
                French_columns = [col for col in hyundai_master_sheet_df.columns if "(FR)" in col]

                # Initialize nested dict if not exists
                key = (year, model, trim_name)
                year_model_trim_dict.setdefault(key, {})

                # Remove English columns for French version and vice versa
                # first check if there are any columns to drop

                
                if len(English_columns) > 0:
                    
                    year_model_trim_dict[key]["FR"] = subset_df.drop(columns=English_columns) # drop English columns

                    # standardize column names for French version
                    year_model_trim_dict[key]["FR"].columns = standard_column_names
                    
                    # get the model_number of the vehicle and add it to the dataframe
                    # then drop some columns to match rate importer format
                    # then group all the df for each model and year and finally save them to an excel files
    
                if len(French_columns) > 0:
                    year_model_trim_dict[key]["EN"] = subset_df.drop(columns=French_columns)
                    
                    # standardize column names for French version
                    year_model_trim_dict[key]["EN"].columns = standard_column_names


                
                # dataset_summary[(year, model, trim_name)] = len(subset_df)
                dataset_summary.loc[len(dataset_summary)] = [year, model, trim_name, len(subset_df)]

                print(f"Year: {year}, Model: {model}, Trim: {trim_name}, Records: {len(subset_df)}")

Year: 2024, Model: Palisade, Trim: Pref, Records: 90
Year: 2024, Model: Palisade, Trim: Urban, Records: 90
Year: 2024, Model: Palisade, Trim: Calli, Records: 93
Year: 2024, Model: Tucson, Trim: Pref, Records: 94
Year: 2024, Model: Tucson, Trim: Trend, Records: 94
Year: 2024, Model: Tucson, Trim: N-Line, Records: 83
Year: 2024, Model: Tucson, Trim: Lux, Records: 89
Year: 2024, Model: Tucson, Trim: Ult HEV, Records: 89
Year: 2024, Model: Tucson, Trim: Ult PHEV, Records: 86
Year: 2024, Model: Santa Cruz, Trim: Pref, Records: 95
Year: 2024, Model: Santa Cruz, Trim: Trend, Records: 95
Year: 2024, Model: Santa Cruz, Trim: Ult, Records: 93
Year: 2024, Model: Elantra, Trim: Ess, Records: 76
Year: 2024, Model: Elantra, Trim: Pref, Records: 79
Year: 2024, Model: Elantra, Trim: Lux, Records: 81
Year: 2024, Model: Elantra, Trim: N-Line, Records: 65
Year: 2024, Model: Elantra, Trim: HEV, Records: 66
Year: 2024, Model: Elantra, Trim: N, Records: 61
Year: 2024, Model: Ioniq 6, Trim: Pref, Records: 43

In [43]:
dataset_summary[dataset_summary["Model"]=='Elantra']["Trim"].unique().tolist()
dataset_summary["Model"].unique()

array(['Palisade', 'Tucson', 'Santa Cruz', 'Elantra', 'Ioniq 6', 'Kona',
       'Kona EV', 'Venue', 'Ioniq 5', 'GV60', 'GV70', 'GV70 EV', 'G70',
       'G80 EV', 'G80', 'G90', 'GV80', 'Sonata', 'Santa Fe', 'GV80 Coupe',
       'Ioniq 9'], dtype=object)

['PACW7K3DULC7',
 'PACW7K3DULCB',
 'PACW7K3DULCN',
 'PA1LAA38B477',
 'PA1LAA38B478',
 'PA1LAA38B4BJ']

In [33]:
dataset_summary

,Year,Model,Trim,Record_Count
0,2024,Palisade,Pref,90
1,2024,Palisade,Urban,90
2,2024,Palisade,Calli,93
3,2024,Tucson,Pref,94
4,2024,Tucson,Trend,94
...,...,...,...,...
193,2026,GV80 Coupe,3.5T e-SC Prestige Black,45
194,2026,Ioniq 9,Ess,70
195,2026,Ioniq 9,Pref,73
196,2026,Ioniq 9,Lux,74
